In [9]:

import urllib.request
from pathlib import Path
from typing import List, NamedTuple, Optional
import os
import pandas as pd
from datetime import  datetime

import cv2

import numpy as np

import streamlit as st

import base64

##
import face_recognition as face_rec
import cv2
import shutil
path = 'employee images'
employeeImg = []

employeeName = []
myList = os.listdir(path)
filename = 'click'
FRAME_WINDOW = st.image([])
cam = cv2.VideoCapture(0) 


st.session_state['loggedIn'] = False

def resize(img, size) :
    width = int(img.shape[1]*size)
    height = int(img.shape[0] * size)
    dimension = (width, height)
    return cv2.resize(img, dimension, interpolation= cv2.INTER_AREA)



def findEncoding(images) :
    imgEncodings = []
    for img in images :
        img = resize(img, 0.50)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encodeimg = face_rec.face_encodings(img)[0]
        imgEncodings.append(encodeimg)
    return imgEncodings
def MarkAttendence(name):
    with open('attendence.csv', 'r+') as f:
        myDatalist =  f.readlines()
        nameList = []
        for line in myDatalist :
            entry = line.split(',')
            nameList.append(entry[0])

        if name not in nameList:
            now = datetime.now()
            timestr = now.strftime('%H:%M')
            f.writelines(f'\n{name}, {timestr}')
            statment = str('welcome to seasia' + name)

for cl in myList :
    curimg = cv2.imread(f'{path}/{cl}')
    employeeImg.append(curimg)
    employeeName.append(os.path.splitext(cl)[0])

EncodeList = findEncoding(employeeImg)

import pandas as pd
import sqlite3
from datetime import datetime

# datetime object containing current date and time
def employee_creation(serial):
    data_base = 'face_rec_attendence'
    serial = 'ID_' + str(serial)
    con = sqlite3.connect(f'{data_base}.db')
    cur = con.cursor()


    cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
    listof_table = (cur.fetchall())
    

    z = []
    for s in listof_table:
        z.append(s[0])

    if serial not in z:
       
        try:  

            cur.execute(f"CREATE TABLE {serial}(Status text,time TIMESTAMP, image_sting text)")
            con.commit()
            con.close()
                   
        except:
            pass

#detect employee
def employee_attendance(serial,status,image_string):
    
    serial = int(serial)    
    
    data_base = 'face_rec_attendence'
    
    now = datetime.now()
    time = now.strftime("%d/%m/%Y %H:%M:%S")
   
    seriall = 'ID_' + str(serial)
    con = sqlite3.connect(f'{data_base}.db')
    cur = con.cursor()
    cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
    listof_table = (cur.fetchall())
    print(listof_table)
    df = employee_fetch(serial)
    if df['Status'].iloc[-1]!=status:

       
        params = (status,time, image_string)



        cur.execute(f"INSERT INTO {seriall} VALUES (?,?, ?)", params)
    con.commit()
    con.close()


#fetch employee
def employee_fetch(serial):
    data_base = 'face_rec_attendence'
    serial = 'ID_' + str(serial)
    con = sqlite3.connect(f'{data_base}.db')
    cur = con.cursor()


    df = pd.read_sql_query(f'SELECT * FROM {serial}', con)
    print(df)
    return df


employee_fetch(3291)


def main():
    st.header("Attendence")

    pages = {

        "Attendence ": employ_recog,        
        "Admin ": Admin,       
    }
    page_titles = pages.keys()

    page_title = st.sidebar.selectbox(
        "Choose the app mode",
        page_titles,
    )
    st.subheader(page_title)

    page_func = pages[page_title]
    page_func()

    st.sidebar.markdown(
        """
---
    """,  # noqa: E501
        unsafe_allow_html=True,
    )







def emprec(img):    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    
    facesInFrame = face_rec.face_locations(img)

    encodeFacesInFrame = face_rec.face_encodings(img, facesInFrame)


    for encodeFace, faceloc in zip(encodeFacesInFrame, facesInFrame) :
        matches = face_rec.compare_faces(EncodeList, encodeFace)
        facedis = face_rec.face_distance(EncodeList, encodeFace)
        print(facedis)
        if min(facedis) < 0.5:
            matchIndex = np.argmin(facedis)

            print(matchIndex)


            name = employeeName[matchIndex].upper()
            bs4str = base64.b64encode(img)
#             y1, x2, y2, x1 = faceloc
#             y1, x2, y2, x1 = y1*4, x2*4, y2*4, x1*4

#             cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 3)
#             cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), cv2.FILLED)
#             cv2.putText(img, name, (x1+6, y2-6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)

            top, right, bottom, left = faceloc
            cv2.rectangle(img, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(img, name,  (left + 6, bottom - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 1)
            name = name.replace(' ','') 
            employee_attendance(name,'IN',bs4str)   
    return img   







        
def Admin():

    if st.session_state['loggedIn'] == False:
        user = st.text_input('Username')
        passwd = st.text_input('Password',type='password')
        if st.button('Login') :


            if user == 'Beastinblack' and passwd == 'AkashRaj' :
                st.session_state['loggedIn'] = True

                st.success("Logged In as {}".format(user))


                # Tasks For Only Logged In Users
                def callback():
                    st.session_state.button_clicked = True

                if st.button('check attendence', on_click=callback):
                    employee_seriel_no  = st.number_input('Employee Seriel no')
                    if st.button('Show attendence'):
                    
                        df = employee_fetch(employee_seriel_no)
                        st.dataframe(df)

                if st.button('Add User'):
                    employee_seriel_no  = st.number_input('Employee Seriel no')
                    img_file_buffer = st.file_uploader("Upload Image of Employee", type=["png","jpg","jpeg"])


                    if st.button('Next'):
                        if img_file_buffer is not None:
                            bytes_data = img_file_buffer.getvalue()
                            cv2_img = cv2.imdecode(np.frombuffer(bytes_data, np.uint8), cv2.IMREAD_COLOR)  
                            cv2_img = cv2.cvtColor(cv2_img, cv2.COLOR_BGR2RGB)
                            img = np.array(cv2_img)
                            #bs4str = base64.b64encode(img)
                            st.image(img)
                            if st.button('Register Employee'):
                                cv2.imwrite(f'employee images/{employee_seriel_no}.jpg', img)
                                employee_creation(employee_seriel_no)
                                
                                st.success(f"{employee_seriel_no} Registered")
           
    
    
    




    
def employ_recog():
        st.title("Webcam Application")
        run = st.checkbox('Run')
        while run:
            ret, frame = cam.read()         
            img = emprec(frame)
            
            FRAME_WINDOW.image(img)













if __name__ == "__main__":
    import os



    main()


SyntaxError: invalid syntax (3568506098.py, line 101)

In [11]:
import pandas as pd
import sqlite3
from datetime import datetime

# datetime object containing current date and time
def employee_creation(serial):
    data_base = 'face_rec_attendence'
    serial = 'ID_' + str(serial)
    con = sqlite3.connect(f'{data_base}.db')
    cur = con.cursor()


    cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
    listof_table = (cur.fetchall())
    

    z = []
    for s in listof_table:
        z.append(s[0])

    if serial not in z:
       
        try:  

            cur.execute(f"CREATE TABLE {serial}(Status text,time TIMESTAMP, image_sting text)")
            con.commit()
            con.close()
                   
        except:
            pass

#detect employee
def employee_attendance(serial,status,image_string):
    
    serial = int(serial)    
    
    data_base = 'face_rec_attendence'
    
    now = datetime.now()
    time = now.strftime("%d/%m/%Y %H:%M:%S")
   
    seriall = 'ID_' + str(serial)
    con = sqlite3.connect(f'{data_base}.db')
    cur = con.cursor()
    cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
    listof_table = (cur.fetchall())
    print(listof_table)
    df = employee_fetch(serial)
    if df['Status'].iloc[-1]!=status:

       
        params = (status,time, image_string)



        cur.execute(f"INSERT INTO {seriall} VALUES (?,?, ?)", params)
    con.commit()
    con.close()


#fetch employee
def employee_fetch(serial):
    data_base = 'face_rec_attendence'
    serial = 'ID_' + str(serial)
    con = sqlite3.connect(f'{data_base}.db')
    cur = con.cursor()


    df = pd.read_sql_query(f'SELECT * FROM {serial}', con)
    print(df)
    return df


In [ ]:
import streamlit as st

if "button_clicked" not in st.session_state:
    st.session_state.button_clicked = False

def callback():
    st.session_state.button_clicked = True

if st.button("Add User", on_click=callback):
    employee_seriel_no  = st.number_input('Employee Seriel no')
    img_file_buffer = st.file_uploader("Upload Image of Employee", type=["png","jpg","jpeg"])


    if st.button("Next"):




        if img_file_buffer is not None:
            bytes_data = img_file_buffer.getvalue()
            cv2_img = cv2.imdecode(np.frombuffer(bytes_data, np.uint8), cv2.IMREAD_COLOR)  
            cv2_img = cv2.cvtColor(cv2_img, cv2.COLOR_BGR2RGB)
            img = np.array(cv2_img)
            #bs4str = base64.b64encode(img)
            st.image(img)
            if st.button('Register Employee'):
                cv2.imwrite(f'employee images/{employee_seriel_no}.jpg', img)
                employee_creation(employee_seriel_no)

                st.success(f"{employee_seriel_no} Registered")
        #st.balloons()




def Admin():

    if st.session_state['loggedIn'] == False:
        user = st.text_input('Username')
        passwd = st.text_input('Password',type='password')
        if st.button('Login') :


            if user == 'b' and passwd == 'b' :
                st.session_state['loggedIn'] = True

                st.success("Logged In as {}".format(user))


                # Tasks For Only Logged In Users
                
                if st.button('check attendence'):
                    st.session_state['In'] = True
                    #employee_seriel_no  = st.number_input('Employee Seriel no')
                    if st.button('Show attendence'): 
                        df = employee_fetch(employee_seriel_no)
                        st.dataframe(df)

                if st.button('Add User'):
                    employee_seriel_no  = st.number_input('Employee Seriel no')
                    img_file_buffer = st.file_uploader("Upload Image of Employee", type=["png","jpg","jpeg"])


                    if st.button('Next'):
                        if img_file_buffer is not None:
                            bytes_data = img_file_buffer.getvalue()
                            cv2_img = cv2.imdecode(np.frombuffer(bytes_data, np.uint8), cv2.IMREAD_COLOR)  
                            cv2_img = cv2.cvtColor(cv2_img, cv2.COLOR_BGR2RGB)
                            img = np.array(cv2_img)
                            #bs4str = base64.b64encode(img)
                            st.image(img)
                            if st.button('Register Employee'):
                                cv2.imwrite(f'employee images/{employee_seriel_no}.jpg', img)
                                employee_creation(employee_seriel_no)
                                
                                st.success(f"{employee_seriel_no} Registered")

In [ ]:
import streamlit as st
import pandas as pd
import sqlite3
from datetime import datetime

# datetime object containing current date and time
def employee_creation(serial):
    data_base = 'face_rec_attendence'
    serial = 'ID_' + str(serial)
    con = sqlite3.connect(f'{data_base}.db')
    cur = con.cursor()


    cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
    listof_table = (cur.fetchall())
    

    z = []
    for s in listof_table:
        z.append(s[0])

    if serial not in z:
       
        try:  

            cur.execute(f"CREATE TABLE {serial}(Status text,time TIMESTAMP, image_sting text)")
            con.commit()
            con.close()
                   
        except:
            pass

#detect employee
def employee_attendance(serial,status,image_string):
    
    serial = int(serial)    
    
    data_base = 'face_rec_attendence'
    
    now = datetime.now()
    time = now.strftime("%d/%m/%Y %H:%M:%S")
   
    seriall = 'ID_' + str(serial)
    con = sqlite3.connect(f'{data_base}.db')
    cur = con.cursor()
    cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
    listof_table = (cur.fetchall())
    print(listof_table)
    df = employee_fetch(serial)
    if df['Status'].iloc[-1]!=status:

       
        params = (status,time, image_string)



        cur.execute(f"INSERT INTO {seriall} VALUES (?,?, ?)", params)
    con.commit()
    con.close()


#fetch employee
def employee_fetch(serial):
    data_base = 'face_rec_attendence'
    serial = 'ID_' + str(serial)
    con = sqlite3.connect(f'{data_base}.db')
    cur = con.cursor()


    df = pd.read_sql_query(f'SELECT * FROM {serial}', con)
    print(df)
    return df
if "button_clicked" not in st.session_state:
    st.session_state.button_clicked = False

def callback():
    st.session_state.button_clicked = True

if st.button("Add User", on_click=callback):

    st.session_state.button_clicked = False
    employee_seriel_no  = st.number_input('Employee Seriel no')
    img_file_buffer = st.file_uploader("Upload Image of Employee", type=["png","jpg","jpeg"])
    


    if st.button("Next", on_click=callback):
        st.session_state.button_clicked = False




        if img_file_buffer is not None:
            bytes_data = img_file_buffer.getvalue()
            cv2_img = cv2.imdecode(np.frombuffer(bytes_data, np.uint8), cv2.IMREAD_COLOR)  
            cv2_img = cv2.cvtColor(cv2_img, cv2.COLOR_BGR2RGB)
            img = np.array(cv2_img)
            #bs4str = base64.b64encode(img)
            st.image(img)
            if st.button('Register Employee', on_click=callback):
                cv2.imwrite(f'employee images/{employee_seriel_no}.jpg', img)
                employee_creation(employee_seriel_no)

                st.success(f"{employee_seriel_no} Registered")

In [ ]:
        col1.header("Original")
        original = Image.open(img0)
        col1.image(original, use_column_width=True)


        col2.header("Grayscale")
        col2.image(img1, use_column_width=True)

In [ ]:
import urllib.request
from pathlib import Path
from typing import List, NamedTuple, Optional
import os
import pandas as pd
from datetime import  datetime
from PIL import Image as im
import cv2

import numpy as np

import streamlit as st

import base64
db_name = 'face_rec_attendence1'
##
import face_recognition as face_rec
import cv2
import shutil
path = 'employee images'
employeeImg = []

employeeName = []
myList = os.listdir(path)
filename = 'click'
FRAME_WINDOW = st.image([])
FRAME_WINDOW1 = st.image([])


col1, col2 = st.columns(2)



cam0 = cv2.VideoCapture(0) 
cam1 = cv2.VideoCapture(1) 

st.session_state['loggedIn'] = False

def resize(img, size) :
    width = int(img.shape[1]*size)
    height = int(img.shape[0] * size)
    dimension = (width, height)
    return cv2.resize(img, dimension, interpolation= cv2.INTER_AREA)



def findEncoding(images) :
    imgEncodings = []
    for img in images :
        img = resize(img, 0.50)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encodeimg = face_rec.face_encodings(img)[0]
        imgEncodings.append(encodeimg)
    return imgEncodings
def MarkAttendence(name):
    with open('attendence.csv', 'r+') as f:
        myDatalist =  f.readlines()
        nameList = []
        for line in myDatalist :
            entry = line.split(',')
            nameList.append(entry[0])

        if name not in nameList:
            now = datetime.now()
            timestr = now.strftime('%H:%M')
            f.writelines(f'\n{name}, {timestr}')
            statment = str('welcome to seasia' + name)

for cl in myList :
    curimg = cv2.imread(f'{path}/{cl}')
    employeeImg.append(curimg)
    employeeName.append(os.path.splitext(cl)[0])

EncodeList = findEncoding(employeeImg)

import pandas as pd
import sqlite3
from datetime import datetime

# datetime object containing current date and time
def employee_creation(serial):
    data_base = db_name
    serial = 'ID_' + str(serial)
    con = sqlite3.connect(f'{data_base}.db')
    now = datetime.now()
    time = now.strftime("%d/%m/%Y %H:%M:%S")
    cur = con.cursor()


    cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
    listof_table = (cur.fetchall())
    
    
    z = []
    for s in listof_table:
        z.append(s[0])

    if serial not in z:
    
        cur.execute(f"CREATE TABLE {serial}(Status text,time TIMESTAMP, image_sting text)")
        params = ('Initial',time, 'Initial_str')



        cur.execute(f"INSERT INTO {seriall} VALUES (?,?, ?)", params)

        con.commit()
        con.close()


#detect employee
def employee_attendance(serial,status,image_string):
    
    serial = int(serial)    
    
    data_base = db_name
    
    now = datetime.now()
    time = now.strftime("%d/%m/%Y %H:%M:%S")
   
    seriall = 'ID_' + str(serial)
    con = sqlite3.connect(f'{data_base}.db')
    cur = con.cursor()
    cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
    listof_table = (cur.fetchall())
    df = employee_fetch(serial)
    if df['Status'].iloc[-1]!=status:

        print(df)
        if df['Status'].iloc[-1]!=status:
            params = (status,time, image_string)



            cur.execute(f"INSERT INTO {seriall} VALUES (?,?, ?)", params)
            st.warning(f'attendance {status} {seriall}')
        con.commit()
        con.close()


#fetch employee
def employee_fetch(serial):
    data_base = db_name
    serial = 'ID_' + str(serial)
    con = sqlite3.connect(f'{data_base}.db')
    cur = con.cursor()


    df = pd.read_sql_query(f'SELECT * FROM {serial}', con)
    print(df)
    return df

st.session_state['loggedIn'] = False
st.session_state['ADD'] = False


def main():
    from PIL import Image
    #image = Image.open('logo.png')
    with open("style.css") as f:
        st.markdown('<style>{}</style>'.format(f.read()), unsafe_allow_html=True)

    path = "C:\Users\vashistanush\Downloads\attendancesystemst\logo.png"
    image = Image.open(path)
    st.image(image, width = 150)

    st.image(image, caption='Welcome to seasia',use_column_width="auto")
    st.title("Mooglelabs Attendance System")

    pages = {

        "Attendance ": employ_recog,        
        "Admin ": Admin,       
    }
    page_titles = pages.keys()

    page_title = st.sidebar.selectbox(
        "Choose the app mode",
        page_titles,
    )

    page_func = pages[page_title]
    page_func()

    st.sidebar.markdown(
        """
---
    """,  # noqa: E501
        unsafe_allow_html=True,
    )
    
    







def emprec0(img0):    
    img0 = cv2.cvtColor(img0, cv2.COLOR_BGR2RGB)
    
    
    facesInFrame = face_rec.face_locations(img0)

    encodeFacesInFrame = face_rec.face_encodings(img0, facesInFrame)


    for encodeFace, faceloc in zip(encodeFacesInFrame, facesInFrame) :
        matches = face_rec.compare_faces(EncodeList, encodeFace)
        facedis = face_rec.face_distance(EncodeList, encodeFace)
        print(facedis)
        if min(facedis) < 0.5:
            matchIndex = np.argmin(facedis)

            print(matchIndex)


            name = employeeName[matchIndex].upper()
            data = im.fromarray(img0)
            data.save("data.jpg")

            image = open('data.jpg', 'rb')
            image_read = image.read()
            bs4str = base64.b64encode(image_read)


            #bs4str = base64.b64encode(img)
#             y1, x2, y2, x1 = faceloc
#             y1, x2, y2, x1 = y1*4, x2*4, y2*4, x1*4

#             cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 3)
#             cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), cv2.FILLED)
#             cv2.putText(img, name, (x1+6, y2-6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)

            top, right, bottom, left = faceloc
            cv2.rectangle(img0, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(img0, name,  (left + 6, bottom - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 1)
            name = name.replace(' ','') 
            employee_attendance(name,'IN',bs4str)   
    return img0   


def emprec1(img1):    
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
    
    
    facesInFrame = face_rec.face_locations(img1)

    encodeFacesInFrame = face_rec.face_encodings(img1, facesInFrame)


    for encodeFace, faceloc in zip(encodeFacesInFrame, facesInFrame) :
        matches = face_rec.compare_faces(EncodeList, encodeFace)
        facedis = face_rec.face_distance(EncodeList, encodeFace)
        print(facedis)
        if min(facedis) < 0.5:
            matchIndex = np.argmin(facedis)

            print(matchIndex)


            name = employeeName[matchIndex].upper()
            data = im.fromarray(img1)
            data.save("data.jpg")

            image = open('data.jpg', 'rb')
            image_read = image.read()
            bs4str = base64.b64encode(image_read)


            #bs4str = base64.b64encode(img)
#             y1, x2, y2, x1 = faceloc
#             y1, x2, y2, x1 = y1*4, x2*4, y2*4, x1*4

#             cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 3)
#             cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), cv2.FILLED)
#             cv2.putText(img, name, (x1+6, y2-6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)

            top, right, bottom, left = faceloc
            cv2.rectangle(img1, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(img1, name,  (left + 6, bottom - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 1)
            name = name.replace(' ','') 
            employee_attendance(name,'OUT',bs4str)   
    return img1   

        
def Admin():

    if st.session_state['loggedIn'] == False:
        user = st.text_input('Username')
        passwd = st.text_input('Password',type='password')
        if st.checkbox('Login') :


            if user == 'b' and passwd == 'b' :
                st.session_state['loggedIn'] = True

                st.success("Logged In as {}".format(user))
                
                # Tasks For Only Logged In Users
                if st.checkbox("ADD_USER"):  
                    st.session_state['ADD'] = True                              
                    employee_seriel_no=st.text_input('Employee Seriel no')
                    st.write('Employee Seriel no ',employee_seriel_no)
                    img_file_buffer = st.file_uploader("Upload Image of Employee", type=["png","jpg","jpeg"])
                    if img_file_buffer is not None:
                        bytes_data = img_file_buffer.getvalue()
                        cv2_img = cv2.imdecode(np.frombuffer(bytes_data, np.uint8), cv2.IMREAD_COLOR)  
                        #cv2_img = cv2.cvtColor(cv2_img, cv2.COLOR_BGR2RGB)
                        img = np.array(cv2_img)
                        cv2.imwrite(f'employee images/{employee_seriel_no}.jpg', img)
                        employee_creation(employee_seriel_no)
                        st.image(img)




                if st.checkbox('check attendence'):
                    st.session_state['In'] = True
                    #employee_seriel_no  = st.number_input('Employee Seriel no')
                    if st.button('Show attendence'): 
                        df = employee_fetch(employee_seriel_no)
                        st.dataframe(df)
    
    
    




    
def employ_recog():
    st.markdown("<h1 style='text-align: center; color: red;'>Webcam Application</h1>", unsafe_allow_html=True)
    run = st.checkbox('Run')
    while run:
        ret0, frame0 = cam0.read()  

        #print(frame)       
        img0 = emprec0(frame0)


        ret1, frame1 = cam1.read()  

        #print(frame)       
        img1 = emprec1(frame1)            
        FRAME_WINDOW.image(img0, use_column_width=True)
        FRAME_WINDOW1.image(img1, use_column_width=True)












if __name__ == "__main__":
    import os



    main()


In [3]:
l= [[1,'b'],[2,3,4,'c','d'],['a','b','c',56],[2]]
for i in l:
    p=[]
    for j in range(len(i)):
        if type(i[j]) == int:
            p.append(i[j])
            break
    for j in range(-1,len(i)):
        if type(i[j]) == str:
            p.append(i[j])
            break
            
    if len(p) == 1:
        if type(p[0]) == int:
            p = None
            
            
        elif type(p[0]) == str:
            p.append(0)
            
            temp = p[0]
            
            temp1 = p[1]
            
            p[0] = temp1
            p[1] = temp0
            
            
    print(p)        
        
        
            
            
            

[1, 'b']
[2, 'd']
[56, 'a']
None
